In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rlway.pyosrd import OSRD
small_infra = OSRD('../cases/small_infra/')

In [3]:
small_infra = OSRD('../cases/small_infra/')

In [6]:
from rlway.pyosrd.viz.map import folium_map
folium_map(small_infra)